In [2]:
from utils.misc import *
from train.train import Train,Test
from datasets.data_utils import load_dataset
from model.Creat_model import creat_model

In [3]:
from torch.backends import cudnn
def fix_seed(seed):
    #seed = 2023
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'   

In [4]:
import networkx
import torch
import scipy.sparse as sp
import numpy as np
import os
import random
from munkres import Munkres
from sklearn import metrics
from sklearn.metrics import adjusted_rand_score as ari_score
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_score
from sklearn.decomposition import PCA
from torch.utils.data import Dataset
import argparse
import yaml
from torch_geometric.utils.convert import to_networkx
import pynvml

def build_args():
    
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('--dataset', type=str, default="acm")
    parser.add_argument('--seed', type=int, default=20)
    parser.add_argument('--cuda', type=bool, default=True)
    parser.add_argument('--n_input', type=int, default=None)
    parser.add_argument('--n_z', type=int, default=None)
    parser.add_argument('--freedom_degree', type=float, default=1.0)
    parser.add_argument('--epoch', type=int, default=None)
    parser.add_argument('--shuffle', type=bool, default=True)
    parser.add_argument('--sigma', type=float, default=None)
    parser.add_argument('--loss_n', type=float, default=None)
    parser.add_argument('--loss_w', type=float, default=None)
    parser.add_argument('--loss_s', type=float, default=None)
    parser.add_argument('--loss_a', type=float, default=None)
    parser.add_argument('--lr', type=float, default=1e-4)
    parser.add_argument('--acc', type=float, default=-1)
    parser.add_argument('--f1', type=float, default=-1)
    args = parser.parse_args([])
    return args

In [5]:
def pca(adata, use_reps=None, n_comps=10):
    
    """Dimension reduction with PCA algorithm"""
    
    from sklearn.decomposition import PCA
    from scipy.sparse.csc import csc_matrix
    from scipy.sparse.csr import csr_matrix
    pca = PCA(n_components=n_comps)
    if use_reps is not None:
       feat_pca = pca.fit_transform(adata.obsm[use_reps])
    else: 
       if isinstance(adata.X, csc_matrix) or isinstance(adata.X, csr_matrix):
          feat_pca = pca.fit_transform(adata.X.toarray()) 
       else:   
          feat_pca = pca.fit_transform(adata.X)
    
    return feat_pca

def clr_normalize_each_cell(adata, inplace=True):
    
    """Normalize count vector for each cell, i.e. for each row of .X"""

    import numpy as np
    import scipy

    def seurat_clr(x):
        # TODO: support sparseness
        s = np.sum(np.log1p(x[x > 0]))
        exp = np.exp(s / len(x))
        return np.log1p(x / exp)

    if not inplace:
        adata = adata.copy()
    
    # apply to dense or sparse matrix, along axis. returns dense matrix
    adata.X = np.apply_along_axis(
        seurat_clr, 1, (adata.X.A if scipy.sparse.issparse(adata.X) else np.array(adata.X))
    )
    return adata     

In [6]:
args = build_args()
args = load_configs(args, "config/configs.yml")
# set_random_seed(2024)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

load configs


In [7]:
import scanpy as sc
adata_omics1 = sc.read_h5ad( '/home/hfzhang/workplace/空间转录组/spaMGCN-main/MGCN-main/datasets/data/humanplacenta/humanplacenta_rna.h5ad')
adata_omics2 = sc.read_h5ad('/home/hfzhang/workplace/空间转录组/spaMGCN-main/MGCN-main/datasets/data/humanplacenta/humanplacenta_ATAC.h5ad')

adata_omics1.var_names_make_unique()
adata_omics2.var_names_make_unique()
adata_omics1 = adata_omics1[adata_omics1.obs['cell_type'].notna()]  
adata_omics2 = adata_omics2[adata_omics2.obs['cell_type'].notna()] 
adata_omics1.obs['final_annot'] = adata_omics1.obs['cell_type'] 
adata_omics2.obs['final_annot'] = adata_omics2.obs['cell_type']

/tmp/ipykernel_29422/196697159.py:9: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_omics1.obs['final_annot'] = adata_omics1.obs['cell_type']
/tmp/ipykernel_29422/196697159.py:10: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_omics2.obs['final_annot'] = adata_omics2.obs['cell_type']


In [8]:

# RNA
sc.pp.filter_genes(adata_omics1, min_cells=10)
sc.pp.highly_variable_genes(adata_omics1, flavor="seurat_v3", n_top_genes=3000)
sc.pp.normalize_total(adata_omics1, target_sum=1e4)
sc.pp.log1p(adata_omics1)
sc.pp.scale(adata_omics1)

adata_omics1 =  adata_omics1[:, adata_omics1.var['highly_variable']]
adata_omics1.obsm['feat'] = pca(adata_omics1, n_comps=adata_omics2.n_vars-1)
# Protein
adata_omics2 = clr_normalize_each_cell(adata_omics2)
sc.pp.scale(adata_omics2)
adata_omics2.obsm['feat'] = pca(adata_omics2, n_comps=adata_omics2.n_vars-1)

/tmp/ipykernel_29422/529664883.py:6: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace is deprecated.
  from scipy.sparse.csc import csc_matrix
/tmp/ipykernel_29422/529664883.py:7: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  from scipy.sparse.csr import csr_matrix
/tmp/ipykernel_29422/529664883.py:6: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace is deprecated.
  from scipy.sparse.csc import csc_matrix
/tmp/ipykernel_29422/529664883.py:7: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  from scipy.sparse.csr import csr_matrix


In [9]:
from utils.preprocess import *
from utils.utils import *

2024-12-30 17:15:32.242865: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-30 17:15:32.382386: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-30 17:15:33.117153: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:
2024-12-30 17:15:33.117241: W tensorflow/compiler/xla/stream_executor/p

In [10]:
def create_adj(adata):
    cell_position_omics1 = adata.obsm['spatial']
    adj_omics1 = construct_graph_by_coordinate(cell_position_omics1, n_neighbors=3)
    adata.uns['adj_spatial'] = adj_omics1
    adj_spatial_omics1 = adata.uns['adj_spatial']
    adj_spatial_omics1 = transform_adjacent_matrix(adj_spatial_omics1)
    adj_spatial_omics1 = adj_spatial_omics1.toarray()
    adj_spatial_omics1 = adj_spatial_omics1 + adj_spatial_omics1.T
    adj_spatial_omics1 = np.where(adj_spatial_omics1>1, 1, adj_spatial_omics1)
    adj = preprocess_graph(adj_spatial_omics1)
    return adj

In [11]:
label=adata_omics2.obs['final_annot'].values
classes, label = np.unique(label, return_inverse=True)
classes = classes.tolist()

In [12]:
args.n_input=adata_omics2.n_vars-1
args.n_input1=adata_omics2.n_vars-1
args.n_clusters=len(classes)
args.n_clusters
args.Type='humanplacenta'
# args.name='Human_tonsil2'
if args.Type == 'Spots':
    args.epoch=200
    args.loss_n=0.01
elif args.Type == '10x':
    args.loss_n=0.001
    args.epoch=300
elif args.Type == 'Stereo-CITE-seq':
    args.loss_n=0.001
    args.epoch=800
elif args.Type == 'humanplacenta':
    args.loss_n=0.01
    args.epoch=300
args.n_clusters=len(classes)

In [13]:
set_random_seed(2021)

In [14]:
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
import anndata
# from SpatialGlue_cv.preprocess import construct_neighbor_graph
# from SpatialGlue_cv.SpatialGlue_pyG import Train_SpatialGlue
# from SpatialGlue_cv.utils import clustering
crossFold=5
kf = KFold(n_splits=crossFold, shuffle=True, random_state=42)
nmis_test=[]
aris_test=[]
amis_test=[]
data=[]
for fold, (train_indices, val_indices) in enumerate(kf.split(adata_omics1)):
    con_loss=[]
    re_loss=[]
    results=[]
    adata_omics1_train, adata_omics1_test = adata_omics1[train_indices], adata_omics1[val_indices]
    adata_omics2_train,adata_omics2_test =adata_omics2[train_indices], adata_omics2[val_indices]
    label_train,label_test=label[train_indices],label[val_indices]
    adj_train=create_adj(adata_omics1_train)
    adj_test=create_adj(adata_omics1_test)
    adj_train = adj_train.to(device)
    adj_test = adj_test.to(device)
    features_omics1_train = torch.FloatTensor(adata_omics1_train.obsm['feat'].copy()).to(device)
    features_omics2_train = torch.FloatTensor(adata_omics2_train.obsm['feat'].copy()).to(device)
    features_omics1_test = torch.FloatTensor(adata_omics1_test.obsm['feat'].copy()).to(device)
    features_omics2_test = torch.FloatTensor(adata_omics2_test.obsm['feat'].copy()).to(device)

    model = creat_model('spamgcn', args).to(device)
    # model=preTrain(200, model, features_omics1_train,features_omics2_train, adj_train, label_train, device, args)
    model=Train(args.epoch, model, features_omics1_train,features_omics2_train, adj_train, label_train, device, args)
    nmi_test,ari_test,ami_test=Test(model,adata_omics1_test,features_omics1_test,features_omics2_test,adj_test,label_test,device,args,'kmeans')
    data.append(adata_omics1_test)
    aris_test.append(ari_test)
    nmis_test.append(nmi_test)
    amis_test.append(ami_test)
nmi=sum(nmis_test)/(fold+1) 
ari=sum(aris_test)/(fold+1)
ami=sum(amis_test)/(fold+1)
print('nmi为{},ari为{},ami{}'.format(nmi,ari,ami))

/tmp/ipykernel_29422/3021765996.py:4: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['adj_spatial'] = adj_omics1
/tmp/ipykernel_29422/3021765996.py:4: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['adj_spatial'] = adj_omics1


  9 loss: 19.327810287475586
Epoch_  9 :acc 0.6825 , nmi 0.6424 , ari 0.5420 , f1 0.5888
 19 loss: 19.287639617919922
Epoch_ 19 :acc 0.6275 , nmi 0.6320 , ari 0.5156 , f1 0.5164
 29 loss: 19.244958877563477
Epoch_ 29 :acc 0.6321 , nmi 0.6544 , ari 0.5324 , f1 0.5089
 39 loss: 19.26008415222168
Epoch_ 39 :acc 0.6381 , nmi 0.6679 , ari 0.5465 , f1 0.5048
 49 loss: 19.191936492919922
Epoch_ 49 :acc 0.7487 , nmi 0.7384 , ari 0.6798 , f1 0.5845
 59 loss: 19.103479385375977
Epoch_ 59 :acc 0.7035 , nmi 0.7031 , ari 0.6071 , f1 0.5542
 69 loss: 18.989965438842773
Epoch_ 69 :acc 0.6990 , nmi 0.7058 , ari 0.6035 , f1 0.5482
 79 loss: 18.846765518188477
Epoch_ 79 :acc 0.7442 , nmi 0.7347 , ari 0.6738 , f1 0.5788
 89 loss: 18.668302536010742
Epoch_ 89 :acc 0.7216 , nmi 0.7161 , ari 0.6328 , f1 0.5561
 99 loss: 18.44809913635254
Epoch_ 99 :acc 0.7449 , nmi 0.7323 , ari 0.6764 , f1 0.5792
109 loss: 18.1811466217041
Epoch_109 :acc 0.7449 , nmi 0.7299 , ari 0.6791 , f1 0.5754
119 loss: 17.863990783691

/tmp/ipykernel_29422/3021765996.py:4: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['adj_spatial'] = adj_omics1
/tmp/ipykernel_29422/3021765996.py:4: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['adj_spatial'] = adj_omics1


  9 loss: 19.270524978637695
Epoch_  9 :acc 0.7163 , nmi 0.6751 , ari 0.6406 , f1 0.5850
 19 loss: 19.227294921875
Epoch_ 19 :acc 0.7366 , nmi 0.6960 , ari 0.6659 , f1 0.5998
 29 loss: 19.18109893798828
Epoch_ 29 :acc 0.7163 , nmi 0.7040 , ari 0.6422 , f1 0.6073
 39 loss: 19.192903518676758
Epoch_ 39 :acc 0.7509 , nmi 0.7226 , ari 0.6935 , f1 0.6078
 49 loss: 19.12189292907715
Epoch_ 49 :acc 0.7592 , nmi 0.7394 , ari 0.6821 , f1 0.6319
 59 loss: 19.030574798583984
Epoch_ 59 :acc 0.7555 , nmi 0.7339 , ari 0.6714 , f1 0.6300
 69 loss: 18.914310455322266
Epoch_ 69 :acc 0.7517 , nmi 0.7353 , ari 0.6686 , f1 0.6296
 79 loss: 18.76813507080078
Epoch_ 79 :acc 0.7239 , nmi 0.7323 , ari 0.6461 , f1 0.6366
 89 loss: 18.585479736328125
Epoch_ 89 :acc 0.7208 , nmi 0.7348 , ari 0.6455 , f1 0.6322
 99 loss: 18.35870933532715
Epoch_ 99 :acc 0.7239 , nmi 0.7338 , ari 0.6417 , f1 0.6361
109 loss: 18.082805633544922
Epoch_109 :acc 0.7239 , nmi 0.7268 , ari 0.6402 , f1 0.6307
119 loss: 17.755535125732422

/tmp/ipykernel_29422/3021765996.py:4: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['adj_spatial'] = adj_omics1
/tmp/ipykernel_29422/3021765996.py:4: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['adj_spatial'] = adj_omics1


  9 loss: 19.105152130126953
Epoch_  9 :acc 0.7383 , nmi 0.7212 , ari 0.6656 , f1 0.6383
 19 loss: 19.060993194580078
Epoch_ 19 :acc 0.7406 , nmi 0.7360 , ari 0.6747 , f1 0.6336
 29 loss: 19.010894775390625
Epoch_ 29 :acc 0.7120 , nmi 0.7239 , ari 0.6433 , f1 0.6100
 39 loss: 19.016372680664062
Epoch_ 39 :acc 0.7098 , nmi 0.7309 , ari 0.6405 , f1 0.6087
 49 loss: 18.934860229492188
Epoch_ 49 :acc 0.7015 , nmi 0.7375 , ari 0.6443 , f1 0.5938
 59 loss: 18.828567504882812
Epoch_ 59 :acc 0.7677 , nmi 0.7603 , ari 0.6911 , f1 0.6800
 69 loss: 18.691240310668945
Epoch_ 69 :acc 0.7150 , nmi 0.7425 , ari 0.6659 , f1 0.5996
 79 loss: 18.516735076904297
Epoch_ 79 :acc 0.7241 , nmi 0.7373 , ari 0.6364 , f1 0.6123
 89 loss: 18.299978256225586
Epoch_ 89 :acc 0.7180 , nmi 0.7353 , ari 0.6307 , f1 0.6065
 99 loss: 18.038379669189453
Epoch_ 99 :acc 0.7233 , nmi 0.7352 , ari 0.6316 , f1 0.6100
109 loss: 17.733930587768555
Epoch_109 :acc 0.7489 , nmi 0.7426 , ari 0.6765 , f1 0.6434
119 loss: 17.39399909

/tmp/ipykernel_29422/3021765996.py:4: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['adj_spatial'] = adj_omics1
/tmp/ipykernel_29422/3021765996.py:4: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['adj_spatial'] = adj_omics1


  9 loss: 18.520591735839844
Epoch_  9 :acc 0.6526 , nmi 0.6491 , ari 0.5459 , f1 0.5452
 19 loss: 18.481706619262695
Epoch_ 19 :acc 0.6662 , nmi 0.6763 , ari 0.5745 , f1 0.5503
 29 loss: 18.436935424804688
Epoch_ 29 :acc 0.6466 , nmi 0.6773 , ari 0.5663 , f1 0.5479
 39 loss: 18.44830322265625
Epoch_ 39 :acc 0.6662 , nmi 0.6944 , ari 0.5911 , f1 0.5527
 49 loss: 18.375675201416016
Epoch_ 49 :acc 0.6722 , nmi 0.7083 , ari 0.6050 , f1 0.5695
 59 loss: 18.28229522705078
Epoch_ 59 :acc 0.6865 , nmi 0.6985 , ari 0.5986 , f1 0.5837
 69 loss: 18.163076400756836
Epoch_ 69 :acc 0.6992 , nmi 0.7278 , ari 0.6307 , f1 0.5772
 79 loss: 18.0119686126709
Epoch_ 79 :acc 0.6970 , nmi 0.7145 , ari 0.6077 , f1 0.5802
 89 loss: 17.82358741760254
Epoch_ 89 :acc 0.7030 , nmi 0.7192 , ari 0.6158 , f1 0.5828
 99 loss: 17.592119216918945
Epoch_ 99 :acc 0.7767 , nmi 0.7727 , ari 0.7588 , f1 0.6336
109 loss: 17.313356399536133
Epoch_109 :acc 0.7729 , nmi 0.7740 , ari 0.7541 , f1 0.6299
119 loss: 16.9840812683105

/tmp/ipykernel_29422/3021765996.py:4: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['adj_spatial'] = adj_omics1
/tmp/ipykernel_29422/3021765996.py:4: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['adj_spatial'] = adj_omics1


  9 loss: 19.05656623840332
Epoch_  9 :acc 0.6662 , nmi 0.6488 , ari 0.5409 , f1 0.5281
 19 loss: 19.01559829711914
Epoch_ 19 :acc 0.6677 , nmi 0.6549 , ari 0.5405 , f1 0.5342
 29 loss: 18.9682559967041
Epoch_ 29 :acc 0.7293 , nmi 0.7004 , ari 0.6458 , f1 0.5818
 39 loss: 18.97580909729004
Epoch_ 39 :acc 0.7368 , nmi 0.7132 , ari 0.6748 , f1 0.5842
 49 loss: 18.89723014831543
Epoch_ 49 :acc 0.7211 , nmi 0.7087 , ari 0.6247 , f1 0.5959
 59 loss: 18.795400619506836
Epoch_ 59 :acc 0.7128 , nmi 0.7111 , ari 0.6300 , f1 0.5819
 69 loss: 18.664575576782227
Epoch_ 69 :acc 0.7068 , nmi 0.7057 , ari 0.6254 , f1 0.5757
 79 loss: 18.498197555541992
Epoch_ 79 :acc 0.7038 , nmi 0.7099 , ari 0.6268 , f1 0.5737
 89 loss: 18.29073715209961
Epoch_ 89 :acc 0.6970 , nmi 0.7056 , ari 0.6220 , f1 0.5683
 99 loss: 18.0377140045166
Epoch_ 99 :acc 0.6526 , nmi 0.7026 , ari 0.5963 , f1 0.5302
109 loss: 17.738237380981445
Epoch_109 :acc 0.6865 , nmi 0.7054 , ari 0.6186 , f1 0.5593
119 loss: 17.394750595092773
E